<a href="https://colab.research.google.com/github/wose70/HDB_Rental/blob/main/03_rental_decision_support.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project 3: HDB Rental Decision Support System

## Features:
1. Tenant Recommendations
2. Landlord Price Optimization
3. Value-for-Money Scoring
4. Location-based Analysis

## Current Date: 2025-05-09 06:58:54

In [16]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
import warnings
warnings.filterwarnings('ignore')

In [17]:
df = pd.read_csv('/content/RentingOutofFlats2025.csv')

class RentalAdvisor:
    def __init__(self, df, current_date='2025-05-09'):
        self.df = df.copy()
        self.current_date = pd.to_datetime(current_date)
        self.prepare_data()

    def prepare_data(self):
        """Prepare data for analysis"""
        # Convert date
        self.df['rent_approval_date'] = pd.to_datetime(self.df['rent_approval_date'])

        # Calculate value score (normalized rent per room)
        self.df['rooms'] = self.df['flat_type'].str.extract('(\d+)').fillna(1).astype(int)
        self.df['rent_per_room'] = self.df['monthly_rent'] / self.df['rooms']

        # Calculate recency score
        self.df['days_old'] = (self.current_date - self.df['rent_approval_date']).dt.days

        # Calculate town average prices
        self.town_stats = self.df.groupby('town').agg({
            'monthly_rent': ['mean', 'std', 'count'],
            'rent_per_room': 'mean'
        }).round(2)

    def find_similar_units(self, budget, rooms, preferred_towns=None, top_n=5):
        """Find similar rental units based on preferences"""
        query = self.df.copy()

        # Filter by rooms and budget
        query = query[query['rooms'] == rooms]
        query = query[query['monthly_rent'] <= budget * 1.1]  # Allow 10% buffer

        if preferred_towns:
            query = query[query['town'].isin(preferred_towns)]

        # Sort by value score and recency
        query['value_score'] = (budget - query['monthly_rent']) / budget
        query['recency_score'] = 1 / (query['days_old'] + 1)
        query['total_score'] = query['value_score'] * 0.7 + query['recency_score'] * 0.3

        return query.nlargest(top_n, 'total_score')

    def optimize_rental_price(self, town, flat_type, premium_factor=0):
        """Suggest optimal rental price for landlords"""
        relevant = self.df[
            (self.df['town'] == town) &
            (self.df['flat_type'] == flat_type)
        ]

        stats = {
            'mean_price': relevant['monthly_rent'].mean(),
            'median_price': relevant['monthly_rent'].median(),
            'min_price': relevant['monthly_rent'].min(),
            'max_price': relevant['monthly_rent'].max(),
            'recent_avg': relevant[relevant['days_old'] <= 30]['monthly_rent'].mean()
        }

        # Calculate suggested price
        base_price = stats['recent_avg'] if not np.isnan(stats['recent_avg']) else stats['mean_price']
        suggested_price = base_price * (1 + premium_factor)

        return stats, round(suggested_price, 2)

    def calculate_value_score(self, town, flat_type, price):
        """Calculate value-for-money score (0-100)"""
        similar = self.df[
            (self.df['town'] == town) &
            (self.df['flat_type'] == flat_type)
        ]['monthly_rent']

        if len(similar) == 0:
            return None

        mean_price = similar.mean()
        std_price = similar.std()

        # Calculate z-score and convert to 0-100 scale
        z_score = (price - mean_price) / std_price
        value_score = 100 - (scipy.stats.norm.cdf(z_score) * 100)

        return round(value_score, 1)

    def analyze_location(self, town):
        """Analyze rental market for a specific location"""
        town_data = self.df[self.df['town'] == town]

        analysis = {
            'total_listings': len(town_data),
            'avg_price': town_data['monthly_rent'].mean(),
            'price_trend': self._calculate_price_trend(town_data),
            'popular_types': town_data['flat_type'].value_counts().to_dict(),
            'value_score': self._calculate_town_value_score(town)
        }

        return analysis

    def _calculate_price_trend(self, data):
        """Calculate 3-month price trend"""
        recent = data[data['days_old'] <= 90]['monthly_rent'].mean()
        older = data[
            (data['days_old'] > 90) &
            (data['days_old'] <= 180)
        ]['monthly_rent'].mean()

        if pd.isna(recent) or pd.isna(older) or older == 0:
            return 0

        return ((recent - older) / older) * 100

    def _calculate_town_value_score(self, town):
        """Calculate overall value score for a town"""
        town_avg = self.town_stats.loc[town]
        global_avg = self.df['rent_per_room'].mean()

        # Higher score means better value
        value_score = (global_avg - town_avg['rent_per_room']['mean']) / global_avg * 100
        return round(value_score + 100, 1)  # Convert to 0-200 scale

In [18]:
def generate_tenant_recommendations(advisor, budget=2500, rooms=3,
                                  preferred_towns=['TAMPINES', 'BEDOK', 'PASIR RIS']):
    """Generate recommendations for tenants"""
    print(f"=== Rental Recommendations (Budget: ${budget}) ===\n")

    # Find similar units
    recommendations = advisor.find_similar_units(budget, rooms, preferred_towns)

    print("Top Recommendations:")
    for idx, row in recommendations.iterrows():
        value_score = advisor.calculate_value_score(row['town'], row['flat_type'], row['monthly_rent'])
        print(f"\n{row['town']} - {row['flat_type']}")
        print(f"Rent: ${row['monthly_rent']:,.2f}")
        print(f"Value Score: {value_score}/100")
        print(f"Days Listed: {row['days_old']}")

    return recommendations

In [19]:
def optimize_landlord_pricing(advisor, town, flat_type, condition='GOOD'):
    """Generate pricing optimization for landlords"""
    # Define premium factors based on condition
    premium_factors = {
        'EXCELLENT': 0.1,
        'GOOD': 0.05,
        'FAIR': 0,
        'POOR': -0.05
    }

    premium = premium_factors.get(condition.upper(), 0)
    stats, suggested_price = advisor.optimize_rental_price(town, flat_type, premium)

    print(f"=== Pricing Analysis for {flat_type} in {town} ===\n")
    print(f"Condition: {condition}")
    print(f"\nMarket Statistics:")
    print(f"Average Price: ${stats['mean_price']:,.2f}")
    print(f"Recent Average (30 days): ${stats['recent_avg']:,.2f}")
    print(f"Price Range: ${stats['min_price']:,.2f} - ${stats['max_price']:,.2f}")
    print(f"\nSuggested Price: ${suggested_price:,.2f}")

    return suggested_price

In [20]:
def analyze_market_opportunities(advisor):
    """Analyze market opportunities and generate insights"""
    print("=== Market Opportunity Analysis ===\n")

    # Analyze each town
    opportunities = []
    for town in advisor.df['town'].unique():
        analysis = advisor.analyze_location(town)

        opportunities.append({
            'town': town,
            'value_score': analysis['value_score'],
            'price_trend': analysis['price_trend'],
            'listings': analysis['total_listings']
        })

    # Convert to DataFrame and sort
    opp_df = pd.DataFrame(opportunities)

    # Print top opportunities
    print("Top Value-for-Money Areas:")
    print(opp_df.nlargest(5, 'value_score')[['town', 'value_score']])

    print("\nFastest Growing Areas:")
    print(opp_df.nlargest(5, 'price_trend')[['town', 'price_trend']])

    return opp_df

In [22]:
# Initialize the advisor
advisor = RentalAdvisor(df, current_date='2025-05-09')

# Example usage
# For tenants
recommendations = generate_tenant_recommendations(
    advisor,
    budget=2500,
    rooms=3,
    preferred_towns=['TAMPINES', 'BEDOK', 'PASIR RIS']
)

# For landlords
suggested_price = optimize_landlord_pricing(
    advisor,
    town='TAMPINES',
    flat_type='4-ROOM',
    condition='GOOD'
)

# Market analysis
opportunities = analyze_market_opportunities(advisor)
display(opportunities) # Display opportunities as a table

=== Rental Recommendations (Budget: $2500) ===

Top Recommendations:

BEDOK - 3-ROOM
Rent: $490.00
Value Score: 100.0/100
Days Listed: 220

BEDOK - 3-ROOM
Rent: $600.00
Value Score: 99.9/100
Days Listed: 647

TAMPINES - 3-ROOM
Rent: $600.00
Value Score: 99.9/100
Days Listed: 647

BEDOK - 3-ROOM
Rent: $600.00
Value Score: 99.9/100
Days Listed: 859

BEDOK - 3-ROOM
Rent: $650.00
Value Score: 99.9/100
Days Listed: 463
=== Pricing Analysis for 4-ROOM in TAMPINES ===

Condition: GOOD

Market Statistics:
Average Price: $2,807.96
Recent Average (30 days): $nan
Price Range: $700.00 - $4,700.00

Suggested Price: $2,948.36
=== Market Opportunity Analysis ===

Top Value-for-Money Areas:
             town  value_score
17        PUNGGOL        114.0
0      ANG MO KIO        110.1
15  MARINE PARADE        108.2
25         YISHUN        107.3
23      TOA PAYOH        105.4

Fastest Growing Areas:
             town  price_trend
15  MARINE PARADE     7.229915
5   BUKIT PANJANG     5.728692
10        GEY

,town,value_score,price_trend,listings
0,ANG MO KIO,110.1,-0.913352,8546
1,BEDOK,104.7,2.471879,9197
2,BISHAN,99.4,-2.605937,3438
3,BUKIT BATOK,99.4,1.310470,5828
4,BUKIT MERAH,101.8,1.050747,8546
5,BUKIT PANJANG,101.7,5.728692,3858
6,BUKIT TIMAH,81.4,-2.495424,461
7,CENTRAL,92.0,-8.001138,2347
8,CHOA CHU KANG,98.6,2.402558,5503
9,CLEMENTI,101.0,-2.221135,5636


This Rental Decision Support System includes:

**Tenant Recommendations**

Budget-based matching
Value-for-money scoring
Location preferences
Unit comparison

**Landlord Price Optimization**

Market-based pricing
Condition adjustments
Competitive analysis
Trend-based suggestions

**Value Score Analysis**

Price-to-room ratio
Location premium
Market positioning
Historical comparison

**Market Opportunities**

Growth areas
Value hotspots
Supply analysis
Trend identification